In [2]:
import time
from tqdm import tqdm
import numpy as np
import pandas as pd

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

service = Service(executable_path="chromedriver.exe")
options = webdriver.ChromeOptions()


In [4]:
def hashContent(csvfile_):
    scrap=pd.read_csv(csvfile_)    #링크함수로 만든 파일에서 읽음
    contentDF = scrap
    contentDF.drop('검색키워드',axis=1, inplace=True)
    
    contentList = []
    hashList = []
    driver = webdriver.Chrome(service=service, options=options)
    for i in tqdm(contentDF['링크']):
        try:
            driver.get(i)
            driver.switch_to.frame('mainFrame')
            driver.implicitly_wait(2+np.random.random())
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            driver.implicitly_wait(2+np.random.random())
        
            element = driver.find_element(By.CLASS_NAME, 'se-main-container').text.replace('\n',' ').replace('~',' ').replace('-',' ')
            contentList.append(element)
        
            hash_ = driver.find_element(By.CLASS_NAME, 'wrap_tag').find_elements(By.TAG_NAME,'span')
            hash_ = ' '.join([i.text.split('#')[1] for i in hash_ if i.text != ''])
            hashList.append(hash_)
        except Exception:
            contentList.append(None)
            hashList.append(None)
    
    contentDF['내용'] = contentList
    contentDF['해시태그'] = hashList

    driver.quit()
    
    with open('블로그내용.csv', 'a', encoding='utf-8') as f:
        contentDF.to_csv(f, header=False, index=False, encoding='utf-8')


In [5]:
tqdm(hashContent('검색링크.csv'))

100%|████████████████████████████████████████████████████████████████████████████| 8855/8855 [6:51:47<00:00,  2.79s/it]
0it [00:00, ?it/s]